In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
!playwright install

In [4]:
SEASONS = list(range(2016,2024))

In [5]:
DATA_DIR = "data2"
STANDINGS_DIR = os.path.join(DATA_DIR, "Standings")
SCORES_DIR = os.path.join(DATA_DIR, "Scores")

In [6]:
async def get_html(url, selector, sleep=4, retries=3):
    html = None
    for i in range (1, retries+1):
        #We need to pause the scraping for a certain number of seconds, otherwise it would be a high frequency scrapping and the site might ban us
        time.sleep(sleep*i)
        
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            
            continue
        else:
            break
    return html

In [8]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#all_schedule")
        
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com


In [10]:
standings_files = os.listdir(STANDINGS_DIR)
print(standings_files)

['NBA_2016_games-april.html', 'NBA_2016_games-december.html', 'NBA_2016_games-february.html', 'NBA_2016_games-january.html', 'NBA_2016_games-june.html', 'NBA_2016_games-march.html', 'NBA_2016_games-may.html', 'NBA_2016_games-november.html', 'NBA_2016_games-october.html', 'NBA_2017_games-april.html', 'NBA_2017_games-december.html', 'NBA_2017_games-february.html', 'NBA_2017_games-january.html', 'NBA_2017_games-june.html', 'NBA_2017_games-march.html', 'NBA_2017_games-may.html', 'NBA_2017_games-november.html', 'NBA_2017_games-october.html', 'NBA_2018_games-april.html', 'NBA_2018_games-december.html', 'NBA_2018_games-february.html', 'NBA_2018_games-january.html', 'NBA_2018_games-june.html', 'NBA_2018_games-march.html', 'NBA_2018_games-may.html', 'NBA_2018_games-november.html', 'NBA_2018_games-october.html', 'NBA_2019_games-april.html', 'NBA_2019_games-december.html', 'NBA_2019_games-february.html', 'NBA_2019_games-january.html', 'NBA_2019_games-june.html', 'NBA_2019_games-march.html', 'NBA_

In [11]:
async def scrape_game(standings_file):
    with open(standings_file, "r") as f:
        html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores  = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores  = [f"https://basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+", encoding="utf-8") as f:
            f.write(html)

In [ ]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)